## LLM

In [1]:
import sys
sys.path.insert(0, '../src/')

import matplotlib.pyplot as plt
from tokenizer import TokenizerBPE
from data_handling import normalize_to_ascii, clean_text
from utils import saver, loader
import numpy as np
import re

import os
import pickle as pkl
from tqdm.notebook import tqdm
import json

# disable gpu for testing purposes
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


## CoQA

In [2]:
with open('../corpus/coqa-train-v1.0.json', 'r', encoding='utf-8') as f:
    coqa = json.load(f)

data = coqa["data"]
length = len(data)

In [3]:
context_list = []
question_grouped = [[] for _ in range(length)]
answer_grouped = [[] for _ in range(length)]


for i, sample in tqdm(enumerate(data), total=length):
    context_list.append(clean_text(normalize_to_ascii(sample["story"])))
    qa = sample["questions"]
    for question in qa:
        question_grouped[i].append(clean_text(normalize_to_ascii(question["input_text"])))
    answers = sample["answers"]
    for answer in answers:
        answer_grouped[i].append(clean_text(normalize_to_ascii(answer["input_text"])))

  0%|          | 0/7199 [00:00<?, ?it/s]

In [4]:
saver("../corpus/coqa_raw", [context_list, question_grouped, answer_grouped])

In [8]:
def fused_sqa(story_list, question_grouped_list, answer_grouped_list):
    q ="<q>"
    a = "<a>"
    sos = "<s>"
    eos = "</s>"

    rcw = re.compile(r"\s+")

    corpus_list = []
    for story, question_list, answer_list in tqdm(list(zip(story_list, question_grouped_list, answer_grouped_list))):
        sqa = [sos, normalize_to_ascii(story).lower()]
        for question, answer in zip(question_list, answer_list):
            sqa.append(q)
            sqa.append(normalize_to_ascii(question).lower())
            sqa.append(a)
            sqa.append(normalize_to_ascii(answer).lower())
        sqa.append(eos)
        corpus_list.append("".join(sqa))
        
    return corpus_list

In [11]:
corpus_list = fused_sqa(context_list, question_grouped, answer_grouped)

  0%|          | 0/7199 [00:00<?, ?it/s]

## 